## 学习算法的实现
**前提**
神经网络存在合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程成为学习。学习步骤分为四个步骤：

**STEP1(mini-batch)**
从训练数据中随机选出一部分数据，这部分数据成为mini-batch。我们的不表示减小mini-batch损失函数的值。

**STEP2计算梯度**
为了减小mini-batch损失函数的值，需要求出各个权重参数的梯度。梯度表示损失函数的值减小最多的方向。

**STEP3更新参数**
将权重参数沿梯度方向进行微小更新。

**STEP4重复**
重复步骤1，2，3

因为使用的数据是mini batch数据，所以称为“随机梯度下降法”（stochastic gradient descent， SGD）

## 实现手写数字识别的神经网络

In [33]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

In [34]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

**参数说明**

input_size：输入层的神经元数量（即输入数据的特征维度）。

hidden_size：隐藏层的神经元数量。

output_size：输出层的神经元数量（如分类任务中的类别数）。

weight_init_std：权重初始化的标准差（默认值 0.01），控制初始权重的随机范围。



w1：输入层到隐藏层的权重矩阵，形状为 (input_size, hidden_size)。

使用 高斯分布（正态分布）随机初始化，乘以 weight_init_std 缩小数值范围，避免初始权重过大导致梯度不稳定。

例如：若 input_size=100，hidden_size=50，则 w1 是一个 100×50 的矩阵。

b1：隐藏层的偏置向量，形状为 (hidden_size,)。

初始化为全零向量（常见做法），因为偏置的初始值对模型影响较小。

In [35]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [36]:
x = np.random.rand(100, 784) # 伪输入数据
y = net.predict(x)

In [37]:
x = np.random.rand(100, 784) # 伪输入数据
t = np.random.rand(100, 10) # 伪正确解标签

grads = net.gradient(x, t)
print(grads)

{'W2': array([[-0.18599029, -0.19411179, -0.20398726, -0.18526207, -0.21131236,
        -0.1965524 , -0.18374991, -0.2091961 , -0.1860776 , -0.18612659],
       [-0.18935989, -0.19954559, -0.2089375 , -0.18958162, -0.21753749,
        -0.20192033, -0.1866773 , -0.21442098, -0.19010769, -0.19008901],
       [-0.19271729, -0.20159175, -0.21282875, -0.19150483, -0.22096671,
        -0.20491423, -0.18983141, -0.21707159, -0.1930493 , -0.19318633],
       [-0.20257271, -0.21134351, -0.2220483 , -0.20070949, -0.2307262 ,
        -0.21463609, -0.1983133 , -0.22841498, -0.20196325, -0.20309596],
       [-0.1963308 , -0.20537131, -0.21584813, -0.19605321, -0.22486834,
        -0.2094447 , -0.19419723, -0.22063238, -0.19650143, -0.19668744],
       [-0.17967075, -0.1871939 , -0.19649029, -0.17808099, -0.20373292,
        -0.19057679, -0.17644256, -0.20192818, -0.17866268, -0.18016743],
       [-0.19402141, -0.20453775, -0.21410903, -0.19261116, -0.22224849,
        -0.20706604, -0.19310691, -0.2

In [38]:
x = np.random.rand(100, 784) 
t = np.random.rand(100, 10) 

grads = net.numerical_gradient(x, t)
print(grads)

{'W1': array([[-5.28871680e-05,  2.47562304e-05, -3.22633753e-04, ...,
        -2.23395391e-05,  2.74621001e-04,  6.39053388e-05],
       [-1.01065716e-04,  1.02887898e-04, -6.73135192e-05, ...,
         5.91739324e-05,  8.77545325e-05, -6.80855705e-05],
       [ 4.27823998e-06,  5.31234545e-05, -2.72634213e-04, ...,
        -1.06939282e-04,  1.53501221e-04,  9.63866542e-05],
       ...,
       [ 4.04601774e-05,  5.65589375e-05, -2.24481416e-04, ...,
         1.22293109e-05,  5.24605293e-05, -6.36145137e-06],
       [-6.54866650e-05, -4.26520974e-05, -1.17410668e-04, ...,
        -3.21776161e-06,  2.08183573e-04,  7.81630516e-05],
       [ 2.07529482e-05, -1.69548464e-05, -1.26401356e-05, ...,
         6.81441192e-05, -4.66881378e-05, -8.58471494e-05]]), 'b1': array([-4.38711067e-06,  5.76035664e-06, -1.89035161e-04, -2.51339083e-05,
       -1.42489194e-04,  6.35133990e-05,  1.82168263e-04, -2.79543722e-06,
       -4.08804879e-05,  1.31229816e-04,  3.74217390e-05,  3.51576657e-05,
    